In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df=pd.read_csv("app_data.log") #location of log file

In [3]:
print(df.head(5))

         DATE      TIME APPLICATION
0  12-06-2023  12:05:01    nautilus
1  12-06-2023  12:05:03    nautilus
2  12-06-2023  12:05:04       gedit
3  12-06-2023  12:05:05       gedit
4  12-06-2023  12:05:06       gedit


# Data Cleaning

In [13]:
		"""Converting the data set into the fomat
		   DATE        START_TIME     END_TIME USAGE   APPLICATION
		   11-01-2022  11:49:42       11:50:33    52   thunar     """

# cleaned_app_data="clean_app_data.csv"    #name of file in which w'll store clean data
new_df = pd.DataFrame(columns=["DATE", "START_TIME", "END_TIME","USAGE","APPLICATION"])

def time_difference(time1, time2):
    """This funcation take 2 time as a parameter and returns total time differnce in seconds"""
    format_str = '%H:%M:%S'
    time_obj1 = datetime.strptime(time1, format_str)
    time_obj2 = datetime.strptime(time2, format_str)
    time_diff = time_obj2 - time_obj1
    return int(time_diff.total_seconds())

start_index=0                    #setting the index for the first data in new df and subsequently to set the starting time of application further
# last_row_index = df.index[-1]    #fetches the last row of df
try:
    for index, row in df.iterrows():                                            #iterating through whole data set 
        if (df.loc[index,"APPLICATION"]==df.loc[(index+1),"APPLICATION"]) and time_difference(df.loc[index,"TIME"], df.loc[(index+1),"TIME"]) < 5 and (df.loc[index,"DATE"]==df.loc[(index+1),"DATE"]):
            # If it's a new application or time difference is >= 5 seconds or date is different, add the row to new_df
                pass
        else:
            new_df.loc[len(new_df)] = [df.loc[index,"DATE"],df.loc[start_index,"TIME"],df.loc[index,"TIME"],time_difference(df.loc[start_index,"TIME"], df.loc[index,"TIME"]),df.loc[index,"APPLICATION"]]   #appending data to new df
            start_index=index                                                   #setting the index for the next application start time
except:
    pass

print(new_df.head())
        

         DATE START_TIME  END_TIME  USAGE   APPLICATION
0  12-06-2023   12:05:01  12:05:03      2      nautilus
1  12-06-2023   12:05:03  12:05:07      4         gedit
2  12-06-2023   12:05:07  12:05:08      1      nautilus
3  12-06-2023   12:05:08  12:05:10      2           gjs
4  12-06-2023   12:05:10  12:05:17      7  sublime_text


In [12]:
prev_row = None
for index, row in df.iterrows():
    if prev_row is None or (row["APPLICATION"] != prev_row["APPLICATION"]) or (time_difference(prev_row["TIME"], row["TIME"]) >= 5) or (row["DATE"] != prev_row["DATE"]):
        # If it's a new application or time difference is >= 5 seconds or date is different, add the row to new_df
        new_df.loc[len(new_df)] = [row["DATE"], row["TIME"], row["TIME"], 0, row["APPLICATION"]]
        start_index = index
    else:
        # Update the END_TIME and USAGE columns for the previous row
        new_df.at[len(new_df) - 1, "END_TIME"] = row["TIME"]
        new_df.at[len(new_df) - 1, "USAGE"] = time_difference(new_df.at[len(new_df) - 1, "START_TIME"], row["TIME"])

    # Update the previous row
    prev_row = row
print(new_df.head())

         DATE START_TIME  END_TIME  USAGE   APPLICATION
0  12-06-2023   12:05:01  12:05:03      2      nautilus
1  12-06-2023   12:05:03  12:05:07      4         gedit
2  12-06-2023   12:05:07  12:05:08      1      nautilus
3  12-06-2023   12:05:08  12:05:10      2           gjs
4  12-06-2023   12:05:10  12:05:17      7  sublime_text


# Show the pie chart of the current day computer usage in pie chart